In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [4]:
df = pd.read_csv("../../data/wine.csv")
df.head()

,Wine,Alcohol,Malic.acid,Ash,Acl,Mg,Phenols,Flavanoids,Nonflavanoid.phenols,Proanth,Color.int,Hue,OD,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [8]:
X = df.values[:,1:14]
y = df.values[:,0]

In [9]:
y #There are three types of wine

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3.])

<b>Splitting data into train and test </b>

In [20]:
from sklearn.model_selection import train_test_split,cross_val_score
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.30,random_state=2018)

<b>Data Preprocessing </b>

In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
sc = StandardScaler()
X_std = sc.fit_transform(X)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

<b>In Bagging we can work with different models </b>

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [17]:
clf_lr = LogisticRegression(penalty = 'l2', C= 1000,random_state = 2018)
clf_dt = DecisionTreeClassifier(max_depth=1 , criterion= 'entropy' , random_state = 2018)
clf_knn = KNeighborsClassifier(n_neighbors = 3)

In [18]:
clf_labels = ['LogisticRegression', 'DecisionTreeClassifier', 'KNeighborsClassifier']

In [22]:
for clf , label in zip([clf_lr , clf_dt , clf_knn], clf_labels):
    scores = cross_val_score(estimator = clf , X= X_train_std , y= y_train , cv=10 , scoring = 'accuracy')
    print('Accuracy : %0.2f (+/- %0.2f) [%s]' % (scores.mean(), scores.std(), label))

Accuracy : 0.99 (+/- 0.03) [LogisticRegression]
Accuracy : 0.58 (+/- 0.05) [DecisionTreeClassifier]
Accuracy : 0.94 (+/- 0.06) [KNeighborsClassifier]


<b> Ensembling </b>

In [23]:
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline

In [26]:
pipe_lr = Pipeline([['sc',StandardScaler()],['clf_lr',clf_lr]])
pipe_knn= Pipeline([['sc',StandardScaler()],['clf_knn',clf_knn]])

In [27]:
mv = VotingClassifier(estimators = [('lr',pipe_lr) , ('dt',clf_dt) , ('knn',pipe_knn)], voting = "hard")

In [28]:
mv = mv.fit(X_train, y_train)

In [30]:
mv_score = cross_val_score(estimator= mv , X= X_train , y= y_train , cv=10 , scoring = 'accuracy')


C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, b

In [31]:
print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (mv_score.mean(),mv_score.std(),'Majority voting'))

Accuracy: 0.97 (+/- 0.04) [Majority voting]
